In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict

import data_API

In [2]:
start_data = '2018-04-27-0'
end_data = '2018-04-27-23'

args = data_API.Param(start_data, end_data)
args.get_data()

done
done
done
done
done


In [3]:
bj_air = pd.read_csv(args.path_bj_air)
ld_air = pd.read_csv(args.path_ld_air)
submission = pd.read_csv('D:/Program/dataset/KDD2018/KDD_dataset/sample_submissioin.csv')

In [4]:
bj_stat = list(OrderedDict.fromkeys(bj_air['station_id'].values))
bj_stat

['dongsi_aq',
 'tiantan_aq',
 'guanyuan_aq',
 'wanshouxigong_aq',
 'aotizhongxin_aq',
 'nongzhanguan_aq',
 'wanliu_aq',
 'beibuxinqu_aq',
 'zhiwuyuan_aq',
 'fengtaihuayuan_aq',
 'yungang_aq',
 'gucheng_aq',
 'fangshan_aq',
 'daxing_aq',
 'yizhuang_aq',
 'tongzhou_aq',
 'shunyi_aq',
 'pingchang_aq',
 'mentougou_aq',
 'pinggu_aq',
 'huairou_aq',
 'miyun_aq',
 'yanqin_aq',
 'dingling_aq',
 'badaling_aq',
 'miyunshuiku_aq',
 'donggaocun_aq',
 'yongledian_aq',
 'yufa_aq',
 'liulihe_aq',
 'qianmen_aq',
 'yongdingmennei_aq',
 'xizhimenbei_aq',
 'nansanhuan_aq',
 'dongsihuan_aq']

In [5]:
ld_stat = list(OrderedDict.fromkeys(ld_air['station_id'].values))
ld_stat

['CD1',
 'BL0',
 'GR4',
 'MY7',
 'HV1',
 'GN3',
 'GR9',
 'LW2',
 'GN0',
 'KF1',
 'CD9',
 'CT3',
 'CT2',
 'BX9',
 'BX1',
 'TH4',
 'ST5',
 'RB7',
 'TD5']

In [6]:
def cut_station(s):
    number_index = s.find('#')
    return s[0:number_index]

sub_stat = list(OrderedDict.fromkeys(submission['test_id'].apply(cut_station).values))
sub_stat

['dongsi_aq',
 'tiantan_aq',
 'guanyuan_aq',
 'wanshouxig_aq',
 'aotizhongx_aq',
 'nongzhangu_aq',
 'wanliu_aq',
 'beibuxinqu_aq',
 'zhiwuyuan_aq',
 'fengtaihua_aq',
 'yungang_aq',
 'gucheng_aq',
 'fangshan_aq',
 'daxing_aq',
 'yizhuang_aq',
 'tongzhou_aq',
 'shunyi_aq',
 'pingchang_aq',
 'mentougou_aq',
 'pinggu_aq',
 'huairou_aq',
 'miyun_aq',
 'yanqin_aq',
 'dingling_aq',
 'badaling_aq',
 'miyunshuik_aq',
 'donggaocun_aq',
 'yongledian_aq',
 'yufa_aq',
 'liulihe_aq',
 'qianmen_aq',
 'yongdingme_aq',
 'xizhimenbe_aq',
 'nansanhuan_aq',
 'dongsihuan_aq',
 'CD1',
 'BL0',
 'GR4',
 'MY7',
 'HV1',
 'GN3',
 'GR9',
 'LW2',
 'GN0',
 'KF1',
 'CD9',
 'ST5',
 'TH4']

In [7]:
ld_stat = sub_stat[35:]
ld_stat

['CD1',
 'BL0',
 'GR4',
 'MY7',
 'HV1',
 'GN3',
 'GR9',
 'LW2',
 'GN0',
 'KF1',
 'CD9',
 'ST5',
 'TH4']

In [8]:
def get_time(t):
    hour = int(t[-8:-6])
    return hour


def insert_row(idx, df):
    df_insert = pd.Series([np.nan])
    for i in idx:        
        dfA = df.iloc[:i, ]
        dfB = df.iloc[i:, ]
        
        df = dfA.append(df_insert, ignore_index = True).append(dfB).reset_index(drop = True)

    return df

def fillWithMean(df):
    return df.fillna(df.mean())

In [9]:
air_index = ["PM25_Concentration","PM10_Concentration","O3_Concentration"]

In [10]:
df = pd.DataFrame()


for i in bj_stat:
    sub_air = bj_air[bj_air['station_id']==i].fillna(method='ffill')
    get_air_time = sub_air['time'].apply(get_time).values
    lost_time = list(set(range(24))-set(get_air_time))
    
    sub_air = sub_air[air_index]
    sub_air = sub_air.reset_index(drop=True)
    
    new_sub = insert_row(lost_time, sub_air)
    
    new_sub = new_sub[air_index]
    new_sub = new_sub.fillna(method='ffill')
    new_sub = new_sub.append(new_sub, ignore_index=True)
    
    df = df.append(new_sub[air_index],ignore_index=True)

    

C:\Anaconda\lib\site-packages\pandas\core\frame.py:5175: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  other.index).difference(self.columns).tolist()


In [11]:
df.head()

,PM25_Concentration,PM10_Concentration,O3_Concentration
0,47.0,215.0,51.0
1,35.0,220.0,79.0
2,35.0,220.0,79.0
3,30.0,229.0,119.0
4,38.0,203.0,133.0


In [12]:
df.shape

(1680, 3)

In [13]:
df.isnull().any()

PM25_Concentration    True
PM10_Concentration    True
O3_Concentration      True
dtype: bool

In [14]:
df[df.isnull().values==True].head()

,PM25_Concentration,PM10_Concentration,O3_Concentration
384,NaN,NaN,NaN
384,NaN,NaN,NaN
384,NaN,NaN,NaN
385,NaN,NaN,NaN
385,NaN,NaN,NaN


In [15]:
df = fillWithMean(df)
df.isnull().any()

PM25_Concentration    False
PM10_Concentration    False
O3_Concentration      False
dtype: bool

In [16]:
df[df.isnull().values==True].head()

,PM25_Concentration,PM10_Concentration,O3_Concentration


In [17]:
for i in ld_stat:
    sub_air = ld_air[ld_air['station_id']==i].fillna(method='ffill')
    get_air_time = sub_air['time'].apply(get_time).values
    lost_time = list(set(range(24))-set(get_air_time))
    
    sub_air = sub_air[air_index]
    sub_air = sub_air.reset_index(drop=True)
    
    new_sub = insert_row(lost_time, sub_air)
    
    new_sub = new_sub[air_index]
    new_sub = new_sub.fillna(method='ffill')
    new_sub = new_sub.append(new_sub, ignore_index=True)
    
    df = df.append(new_sub[air_index],ignore_index=True)

In [18]:
df.shape

(2304, 3)

In [19]:
df.isnull().any()

PM25_Concentration    True
PM10_Concentration    True
O3_Concentration      True
dtype: bool

In [20]:
df[1680:] = fillWithMean(df[1680:])
df[1680:].isnull().any()

PM25_Concentration    False
PM10_Concentration    False
O3_Concentration       True
dtype: bool

In [21]:
df.isnull().any()

PM25_Concentration    False
PM10_Concentration    False
O3_Concentration       True
dtype: bool

In [22]:
df[df.values<0] = np.abs(df[df.values<0].values)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in less
  """Entry point for launching an IPython kernel.


In [23]:
rename_cols = {'PM25_Concentration': 'PM2.5', 'PM10_Concentration': 'PM10', 'O3_Concentration':'O3'}

df = df.rename(columns=rename_cols)
df.head()

,PM2.5,PM10,O3
0,47.0,215.0,51.0
1,35.0,220.0,79.0
2,35.0,220.0,79.0
3,30.0,229.0,119.0
4,38.0,203.0,133.0


In [24]:
submission.head()

,test_id,PM2.5,PM10,O3
0,dongsi_aq#0,0,0,0
1,dongsi_aq#1,0,0,0
2,dongsi_aq#2,0,0,0
3,dongsi_aq#3,0,0,0
4,dongsi_aq#4,0,0,0


In [25]:
df['test_id'] = submission['test_id']
df.head()

,PM2.5,PM10,O3,test_id
0,47.0,215.0,51.0,dongsi_aq#0
1,35.0,220.0,79.0,dongsi_aq#1
2,35.0,220.0,79.0,dongsi_aq#2
3,30.0,229.0,119.0,dongsi_aq#3
4,38.0,203.0,133.0,dongsi_aq#4


In [26]:
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.head()

,test_id,PM2.5,PM10,O3
0,dongsi_aq#0,47.0,215.0,51.0
1,dongsi_aq#1,35.0,220.0,79.0
2,dongsi_aq#2,35.0,220.0,79.0
3,dongsi_aq#3,30.0,229.0,119.0
4,dongsi_aq#4,38.0,203.0,133.0


In [27]:
start = int(start_data.split('-')[-2])+1
start = start_data[:8]+str(start)

end = int(end_data.split('-')[-2])+2
end = end_data[:8]+str(end)

In [28]:
res_path = 'D:/Program/dataset/KDD2018/result/'
csv_name = res_path+'result_'+start+'-'+end+'.csv'
df.to_csv(csv_name, index=False)